In [1]:
import os
import copy
import pickle
import sympy
import functools
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from error_injection import MissingValueError, SamplingError, Injector
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.metrics import mutual_info_score, auc, roc_curve, roc_auc_score, f1_score
from scipy.optimize import minimize as scipy_min
from scipy.spatial import ConvexHull
from scipy.optimize import minimize, Bounds, linprog
from sympy import Symbol as sb
from sympy import lambdify
from tqdm.notebook import trange,tqdm
from IPython.display import display,clear_output
from random import choice
from sklearn.linear_model import LinearRegression
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import _tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score

class style():
    RED = '\033[31m'
    GREEN = '\033[32m'
    BLUE = '\033[34m'
    RESET = '\033[0m'

np.random.seed(1)

# ignore all the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_ins_cleaned():
    # fetch dataset
    auto_mpg = pd.read_csv('datasets/insurance.csv').drop('sex', axis=1).drop('smoker', axis=1).drop('region', axis=1).replace('?', np.nan)
    features = ['age', 'bmi', 'children']
    X = auto_mpg[features].astype(float)
    y = auto_mpg['charges']
    
    # assumed gt imputation
    imputer = KNNImputer(n_neighbors=10)
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train = copy.deepcopy(X_train).reset_index(drop=True)
    X_test = copy.deepcopy(X_test).reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    return X_train, X_test, y_train, y_test

X_train_ins, X_test_ins, y_train_ins, y_test_ins = load_ins_cleaned()
len(X_train_ins)

1070

In [3]:
# first impute the data and make it hypothetically clean
def load_mpg_cleaned():
    # fetch dataset
    auto_mpg = pd.read_csv('datasets/auto-mpg.csv').drop('car name', axis=1).replace('?', np.nan)
    
    features = ['cylinders', 'displacement', 'horsepower', 'weight',
                'acceleration', 'model year', 'origin']
    X = auto_mpg[features].astype(float)
    y = auto_mpg['mpg']
    
    # assumed gt imputation
    imputer = KNNImputer(n_neighbors=10)
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train = copy.deepcopy(X_train).reset_index(drop=True)
    X_test = copy.deepcopy(X_test).reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    return X_train, X_test, y_train, y_test
X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg = load_mpg_cleaned()
len(X_train_mpg)

318

In [4]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
boston_df = pd.read_csv('datasets/housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [5]:
#Useful functions
symbol_id = -1
def create_symbol(suffix=''):
    global symbol_id
    symbol_id += 1
    name = f'e{symbol_id}_{suffix}' if suffix else f'e{symbol_id}'
    return sympy.Symbol(name=name)


#scaler_symbols = set([sb(f'k{i}') for i in range(X_train.shape[1]+1)])
#linearization_dict = dict()
#reverse_linearization_dict = dict()

def inject_sensitive_ranges(X, y, uncertain_attr, uncertain_num, boundary_indices, uncertain_radius_pct=None, 
                  uncertain_radius=None, seed=42):
    global symbol_id
    symbol_id = -1
    
    X_extended = np.append(np.ones((len(X), 1)), X, axis=1)
    ss = StandardScaler()
    X_extended[:, 1:] = ss.fit_transform(X_extended[:, 1:])
    X_extended_symb = sympy.Matrix(X_extended)
    
    if not(uncertain_attr=='y'):
        uncertain_attr_idx = X.columns.to_list().index(uncertain_attr) + 1
        if not(uncertain_radius):
            uncertain_radius = uncertain_radius_pct*(np.max(X_extended[:, uncertain_attr_idx])-\
                                                     np.min(X_extended[:, uncertain_attr_idx]))
    else:
        if not(uncertain_radius):
            uncertain_radius = uncertain_radius_pct*(y_train.max()-y_train.min())[0]
    
    np.random.seed(seed)
    uncertain_indices = boundary_indices[:uncertain_num]
    y_symb = sympy.Matrix(y)
    symbols_in_data = set()
    #print(uncertain_indices)
    for uncertain_idx in uncertain_indices:
        new_symb = create_symbol()
        symbols_in_data.add(new_symb)
        if uncertain_attr=='y':
            y_symb[uncertain_idx] = y_symb[uncertain_idx] + uncertain_radius*new_symb
        else:
            X_extended_symb[uncertain_idx, uncertain_attr_idx] = X_extended_symb[uncertain_idx, uncertain_attr_idx] + uncertain_radius*new_symb
    return X_extended_symb, y_symb, symbols_in_data, ss

# if interval=True, use interval arithmetic, otherwise use zonotopes
def compute_robustness_ratio_sensitive_label_error(X_train, y_train, X_test, y_test, robustness_radius,
                                         uncertain_num, boundary_indices, uncertain_radius=None, 
                                         lr=0.1, seed=42, interval=True):
    X, y, symbols_in_data, ss = inject_sensitive_ranges(X=X_train, y=y_train, uncertain_attr='y', 
                                              uncertain_num=uncertain_num, boundary_indices=boundary_indices, 
                                              uncertain_radius=uncertain_radius, 
                                              uncertain_radius_pct=None, seed=seed)
    
    assert len(X.free_symbols)==0
    # closed-form
    param = (X.T*X).inv()*X.T*y
    
    if interval:
        # make param intervals
        for d in range(len(param)):
            expr = param[d]
            if not(expr.free_symbols):
                continue
            else:
                constant_part = 0
                interval_radius = 0
                for arg in expr.args:
                    if arg.free_symbols:
                        interval_radius += abs(arg.args[0])
                    else:
                        assert constant_part == 0
                        constant_part = arg
                param[d] = constant_part + create_symbol()*interval_radius
    
    test_preds = sympy.Matrix(np.append(np.ones((len(X_test), 1)), ss.transform(X_test), axis=1))*param
    robustness_ls = []
    for pred in test_preds:
        pred_range_radius = 0
        for arg in pred.args:
            if arg.free_symbols:
                pred_range_radius += abs(arg.args[0])
        if pred_range_radius <= robustness_radius:
            robustness_ls.append(1)
        else:
            robustness_ls.append(0)
    
#     print(param)
    return np.mean(robustness_ls)

In [6]:
#accuracy r2 = True; rmse, mse = False for maximize
def leave_one_out(X_train, y_train, X_test, y_test, model, metric, maximize=True): 
    predictions = model.fit(X_train, y_train).predict(X_test)
    initial_metric = metric(y_test.to_numpy(), predictions)
    influence_results = []
   
    for i in range(len(X_train)):
        X_train_new = np.delete(X_train, i, axis=0)
        y_train_new = np.delete(y_train, i, axis=0)
       
        model_clone = model.__class__(**model.get_params())
        new_preds = model_clone.fit(X_train_new, y_train_new).predict(X_test)
        new_metric = metric(y_test.to_numpy(), new_preds)
       
        metric_diff = (initial_metric - new_metric) if maximize else (new_metric - initial_metric)
        
        influence_results.append((i, metric_diff))
       
       
   
    influence_results = sorted(influence_results,key=lambda x: x[1], reverse=True)
    #print(influence_results)
    return [i[0] for i in influence_results]

def mae(y_true, y_pred):
    return sum(abs(y_true - y_pred))/len(y_true)

def mse(y_true, y_pred):
    return sum((y_true - y_pred)**2)/len(y_true)

def r_squared(y_true, y_pred):
    y_bar = np.mean(y_true)
    return 1 -(sum((y_true - y_pred)**2)/sum((y_true-y_bar)**2))

def rmse(y_true, y_pred):
    return np.sqrt(sum((y_true - y_pred)**2)/len(y_true))

lr = LinearRegression()

X_train_ins, X_test_ins, y_train_ins, y_test_ins = X_train_ins.reset_index(drop=True) , X_test_ins.reset_index(drop=True) , y_train_ins.reset_index(drop=True) , y_test_ins.reset_index(drop=True)
boundary_indices_ins = leave_one_out(X_train_ins, y_train_ins, X_test_ins, y_test_ins, lr, mse, maximize=False)

X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg = X_train_mpg.reset_index(drop=True) , X_test_mpg.reset_index(drop=True) , y_train_mpg.reset_index(drop=True) , y_test_mpg.reset_index(drop=True)
boundary_indices_mpg = leave_one_out(X_train_mpg, y_train_mpg, X_test_mpg, y_test_mpg, lr, mse, maximize=False)

boundary_indices_lst = [boundary_indices_ins, boundary_indices_mpg]

In [7]:
X = boston_df.drop(columns=['MEDV'])
y = boston_df['MEDV']
X_train_bos, X_test_bos, y_train_bos, y_test_bos = train_test_split(X, y, test_size=0.2, random_state=1)

print("Training data shape:", X_train_bos.shape)
print("Testing data shape:", X_test_bos.shape)

Training data shape: (404, 13)
Testing data shape: (102, 13)


In [8]:
def select_best_feature(X, y, method="correlation"):
    if method == "correlation":
        correlations = X.corrwith(y)
        best_feature = correlations.abs().idxmax()
    return best_feature

best_feature = select_best_feature(X_train_bos, y_train_bos)
best_feature

'LSTAT'

In [9]:
lr = LinearRegression()
X_train_bos, X_test_bos, y_train_bos, y_test_bos = X_train_bos.reset_index(drop=True) , X_test_bos.reset_index(drop=True) , y_train_bos.reset_index(drop=True) , y_test_bos.reset_index(drop=True)
boundary_indices_bos = leave_one_out(X_train_bos, y_train_bos, X_test_bos, y_test_bos, lr, mae, maximize=False)

In [10]:
# Decision Tree research: 1% of the data
array_indexes = np.zeros(len(X_train_bos))
perc = 0.1 * len(X_train_bos)
for i in range(0, len(X_train_bos)):
    if i <= perc:
        index = boundary_indices_bos[i]
        array_indexes[index] = 1

clf = DecisionTreeClassifier(max_depth=None)
clf.fit(X_train_bos, array_indexes)

feature_max_values = X_train_bos.max()

def get_positive_paths(tree, feature_names, node=0, depth=0, conditions=None, results=None, min_positive_ratio=0.5):
    if conditions is None:
        conditions = {}
    if results is None:
        results = []

    left_child = tree.children_left[node]
    right_child = tree.children_right[node]
    threshold = tree.threshold[node]
    feature = tree.feature[node]

    # Count samples in this node
    sample_count = int(tree.n_node_samples[node])
    positive_count = int(tree.value[node][0, 1]) if tree.n_outputs == 1 else int(tree.value[node][0][1])
    negative_count = int(tree.value[node][0, 0]) if tree.n_outputs == 1 else int(tree.value[node][0][0])

    # Calculate the positive ratio for this node
    positive_ratio = positive_count / sample_count if sample_count > 0 else 0

    # If it's a leaf or qualifies as a 'positive node' by ratio, store the path
    if (left_child == _tree.TREE_LEAF and right_child == _tree.TREE_LEAF) or positive_ratio >= min_positive_ratio:
        path_conditions = {}
        for feat, bounds in conditions.items():
            lower_bound = bounds.get('lower', 0)
            upper_bound = bounds.get('upper', feature_max_values.get(feat, '∞'))  # Use the max value for the feature
            path_conditions[feat] = (lower_bound, upper_bound)
        
        # Only store if there are significant positives
        if positive_count > 0:  # Ensure that there's at least one positive sample
            results.append((positive_count, sample_count, path_conditions, positive_ratio, depth))

    # Update bounds for the current feature in conditions and recurse
    feature_name = feature_names[feature] if feature != _tree.TREE_UNDEFINED else None
    if left_child != _tree.TREE_LEAF and feature_name:
        # Left child represents the <= threshold split
        new_conditions = {k: v.copy() for k, v in conditions.items()}
        new_conditions.setdefault(feature_name, {}).update({'upper': threshold})
        get_positive_paths(tree, feature_names, left_child, depth + 1, new_conditions, results, min_positive_ratio)

    if right_child != _tree.TREE_LEAF and feature_name:
        # Right child represents the > threshold split
        new_conditions = {k: v.copy() for k, v in conditions.items()}
        new_conditions.setdefault(feature_name, {}).update({'lower': threshold})
        get_positive_paths(tree, feature_names, right_child, depth + 1, new_conditions, results, min_positive_ratio)

    # Print and store paths after completing all nodes, if we are at the root node
    if node == 0:
        # Sort results first by depth (root to leaf), then by positive ratio, and then by positive count
        top_results = sorted(results, key=lambda x: (x[4], x[3], x[0]), reverse=False)[:3]  # Prioritize by depth first
        
        # Store top thresholds
        top_thresholds = []
        for idx, (pos_count, total_count, conditions, pos_ratio, dep) in enumerate(top_results, start=1):
            top_thresholds.append(conditions)  # Save conditions (thresholds) for each scenario    
        return top_thresholds
            
tree = clf.tree_
feature_names = X_train_bos.columns
best_thresholds = get_positive_paths(tree, feature_names)
thresholds = [thres[best_feature] for thres in best_thresholds if best_feature in thres]
thresholds

[(0, 12.130000114440918)]

In [11]:
def discretize_and_sample(X_train, feature, thresholds, total_samples, num_bins):
    feature_values = X_train[feature]  # Extract the column of interest
    min_val, max_val = feature_values.min(), feature_values.max()

    # Ensure the values are within the valid range of bins (clip out-of-range values)
    feature_values = feature_values.clip(lower=min_val, upper=max_val)

    # Dynamically calculate bin edges based on the feature values range
    bin_edges = np.linspace(min_val, max_val, num_bins + 1)
    bin_counts, bin_labels = pd.cut(feature_values, bins=bin_edges, labels=False, retbins=True)

    selected_bins_by_threshold = []  # Combined threshold list
    for thresh in thresholds:  # Iterate over age thresholds
        selected_bins = []  # New bin list for that respective threshold
        for bin_idx in range(num_bins):  # For each bin
            bin_lower = bin_edges[bin_idx]  # Get lower bin threshold at bin_idx
            bin_upper = bin_edges[bin_idx + 1]  # Get upper bin threshold at bin_idx
            if thresh[0] <= bin_lower and thresh[1] >= bin_upper:  # If bin meets threshold requirements
                selected_bins.append(bin_idx)
        selected_bins_by_threshold.append(selected_bins)  # Add threshold bin list to combined threshold list
    sampled_indices = set()

    for threshold_idx, selected_bins in enumerate(selected_bins_by_threshold):
        bin_freqs = feature_values.groupby(bin_counts).size()  # Calculate bin frequency of each bin for this threshold
        
        # Reindex to ensure all bins are accounted for, including those with zero frequency
        bin_freqs = bin_freqs.reindex(range(num_bins), fill_value=0)
        bin_priority = sorted([(bin_idx, bin_freqs[bin_idx])  # Sort by frequency of bin, decreasing
                           for bin_idx in selected_bins], 
                          key=lambda x: -x[1]) 
        
        for bin_idx, _ in bin_priority:  # Enumerate over the bins ordered by frequency for this threshold
            bin_indices = feature_values[bin_counts == bin_idx].index  # Get the indices
            needed = total_samples - len(sampled_indices)  # Get however many values are still needed to grab
            
            for idx in bin_indices:  # Iterate over indices
                if len(sampled_indices) < total_samples:
                    sampled_indices.add(idx)
                else:
                    break
            if len(sampled_indices) >= total_samples:
                break

        if len(sampled_indices) >= total_samples:  # End early if total samples needed is met
            break

    #print("Selected bins:", selected_bins_by_threshold)
    #print("Sampled indices:", sampled_indices)
            
    return list(sampled_indices), bin_edges

boundary_indices_bos, bin_edges = discretize_and_sample(X_train_bos, best_feature, thresholds, total_samples=int(0.1 * len(X_train_bos)), num_bins=32)
boundary_indices_bos_heau, bin_edges = discretize_and_sample(X_train_bos, best_feature, thresholds, total_samples=int(0.1 * len(X_train_bos)), num_bins=7)

In [12]:
uncertain_radius_ins = 0.1*(y_train_ins.max() - y_train_ins.min())
uncertain_radius_mpg = 0.1*(y_train_mpg.max() - y_train_mpg.min())
uncertain_radius_bos = 0.1*(y_train_bos.max() - y_train_bos.min())
uncertain_radii = [uncertain_radius_ins, uncertain_radius_mpg, uncertain_radius_bos, uncertain_radius_bos]

uncertain_percentage = 0.1
uncertain_num_ins = int(uncertain_percentage*len(y_train_ins))
uncertain_num_mpg = int(uncertain_percentage*len(y_train_mpg))
uncertain_num_bos = int(uncertain_percentage*len(y_train_bos))
uncertain_numbers = [uncertain_num_ins, uncertain_num_mpg, uncertain_num_bos, uncertain_num_bos]

dataset_sizes = [len(y_train_ins), len(y_train_mpg), len(y_train_bos), len(y_train_bos)]
dataset_names = ["Insurance", "MPG", "BOS", "BOS"]

dataset_dct = {}
dataset_dct["Insurance"] = [X_train_ins, X_test_ins, y_train_ins, y_test_ins]
dataset_dct["MPG"] = [X_train_mpg, X_test_mpg, y_train_mpg, y_test_mpg]
dataset_dct["BOS"] = [X_train_bos, X_test_bos, y_train_bos, y_test_bos]

In [13]:
boundary_indices_lst.append(boundary_indices_bos)
boundary_indices_lst.append(boundary_indices_bos_heau)

In [15]:
def robustness_score_normalization(uncertain_numbers, uncertain_radii, dataset_sizes, boundary_indices_lst, dataset_names, dataset_dct):
    robustness_radii_10 = [] #find robustness radius that grants radii robustness ratio of 0.10 or more 
                             #(alt. use 0.5 instead {depending on closeness, this ratio may need to be larger})

    for i in range(0, len(dataset_names)):
        uncertain_number = uncertain_numbers[i]
        uncertain_radius = uncertain_radii[i]
        boundary_indices = boundary_indices_lst[i]
        X_train, X_test, y_train, y_test = dataset_dct[dataset_names[i]]
        
        robustness_radius=1
        radius_increment = 3

        robustness_ratio = compute_robustness_ratio_sensitive_label_error(X_train, y_train, X_test, y_test, 
                                                                    uncertain_num=uncertain_number,
                                                                    boundary_indices=boundary_indices,
                                                                    uncertain_radius=uncertain_radius, 
                                                                    robustness_radius=robustness_radius,
                                                                    interval=False)
        
        with tqdm(total=500, desc=f"Finding radius for {dataset_names[i]}", leave=False) as pbar:
            while robustness_ratio < 0.5:
                robustness_radius += radius_increment
                robustness_ratio = compute_robustness_ratio_sensitive_label_error(X_train, y_train, X_test, y_test, 
                                                                    uncertain_num=uncertain_number,
                                                                    boundary_indices=boundary_indices,
                                                                    uncertain_radius=uncertain_radius, 
                                                                    robustness_radius=robustness_radius,
                                                                    interval=False)
                pbar.update(radius_increment)
        
        robustness_radii_10.append(robustness_radius)

    results = {}
    for i, dataset_name in enumerate(dataset_names):
        normalized_radius = (1 - (robustness_radii_10[i]/max(robustness_radii_10)))
        normalized_size = (dataset_sizes[i]/max(dataset_sizes))
        robustness_score = 0.5*normalized_radius + 0.5*normalized_size        
        print(f"Normalized robustness score for {dataset_name} dataset is {robustness_score:.4f}")
        results[dataset_name] = robustness_score
    return results

robustness_score_normalization(uncertain_numbers, uncertain_radii, dataset_sizes, boundary_indices_lst, dataset_names, dataset_dct)

Finding radius for Insurance:   0%|          | 0/500 [00:00<?, ?it/s]

Finding radius for MPG:   0%|          | 0/500 [00:00<?, ?it/s]

Finding radius for BOS:   0%|          | 0/500 [00:00<?, ?it/s]

Finding radius for BOS:   0%|          | 0/500 [00:00<?, ?it/s]

Normalized robustness score for Insurance dataset is 0.5000
Normalized robustness score for MPG dataset is 0.6481
Normalized robustness score for BOS dataset is 0.6868
Normalized robustness score for BOS dataset is 0.6868


{'Insurance': 0.5, 'MPG': 0.6480966263275809, 'BOS': 0.6867790286748094}

Normalized robustness score for Insurance dataset is 0.5000

Normalized robustness score for MPG dataset is 0.6474

{'Insurance': 0.5, 'MPG': 0.6473962077641984}

In [ ]:
robustness_ratio = compute_robustness_ratio_sensitive_label_error(X_train_mpg, y_train_mpg, X_test_mpg, y_test_mpg, 
                                                                    uncertain_num=uncertain_num_mpg,
                                                                    boundary_indices=boundary_indices_mpg,
                                                                    uncertain_radius=uncertain_radius_mpg, 
                                                                    robustness_radius=1, 
                                                                    interval=False)

In [ ]:
dataset_sizes

In [ ]:
lst = []
lst.append(5)

In [ ]:
lst

In [ ]:
max(lst)